In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import stilus.models as m
import pytorch_lightning as pl
from stilus.data.sets import MidiDataset
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
print("PyTorch version:", torch.__version__)
print("Lightning version:", pl.__version__)

In [ ]:
cuda_available =  torch.cuda.is_available()
device = torch.device('cuda' if cuda_available else 'cpu')
print('Using device:', device)

#nets = [m.ConvNet_1_0_0("."), m.ConvNet_1_0_1("."), m.ConvNet_1_0_2("."), m.ConvNet_1_0_3("."),m.TransformerNet_1_0_0("."),m.TransformerNet_1_0_1("."),m.TransformerNet_1_0_2(".")]
#best nets seem to be both versions of 1.0.2
nets = [m.ConvNet_1_0_0()]

print(nets)

In [ ]:
# Test that tensor shapes are correct
for net in nets:
    net.set_data_path("data/bach")
    print(net.name)
    input = torch.randn(128, 5, 64)
    out = net(input)
    print("input:", input.shape)
    print("output:", out.shape)

In [ ]:
for net in nets:    
    early_stopping = EarlyStopping('val_loss')
    tb_logger = TensorBoardLogger("tb_logs", name=net.name)
    
    print('Starting to train:', net.name)
    trainer = pl.Trainer(min_epochs=30, max_epochs=400, gpus=1, callbacks=early_stopping , logger=tb_logger)
    trainer.fit(net)
    #trainer.test(net)
    print('Finished training', net.name)